In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [12]:
# RBF Kernel function
def rbf_kernel(x1, x2, gamma=0.1):
    # Calculating the squared Euclidean distance between x1 and x2
    dist_sq = torch.cdist(x1, x2) ** 2  # squared Euclidean distance
    return torch.exp(-gamma * dist_sq)

In [13]:
# Custom Neural Network class with Kernel-based activation
class KernelNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(KernelNN, self).__init__()
        
        # Define the layers (weights are initialized automatically)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Apply the RBF kernel to the input and weights of the first layer
        hidden_output = rbf_kernel(x, self.fc1.weight)  # x is (batch_size, input_size), self.fc1.weight is (hidden_size, input_size)
        
        # Apply bias for the hidden layer output
        hidden_output = hidden_output + self.fc1.bias.view(1, -1)
        
        # Apply the RBF kernel to the hidden layer output and weights of the second layer
        output = rbf_kernel(hidden_output, self.fc2.weight)
        
        # Apply bias for the output layer
        output = output + self.fc2.bias.view(1, -1)
        
        return output

In [14]:
# Sample dataset (for example, using a random dataset)
X = torch.randn(10, 5)  # 10 samples, 5 features
y = torch.randint(0, 2, (10, 1))  # Binary target (0 or 1)

In [15]:
# Instantiate the neural network
model = KernelNN(input_size=5, hidden_size=3, output_size=1)

In [16]:
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [17]:
# Training loop (for simplicity, we’ll just do one iteration)
for epoch in range(100):
    optimizer.zero_grad()  # Reset gradients
    
    # Forward pass
    outputs = model(X)
    
    # Compute the loss
    loss = criterion(outputs, y.float())
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item()}')

Epoch [1/100], Loss: 0.3554430603981018
Epoch [11/100], Loss: 0.31897324323654175
Epoch [21/100], Loss: 0.29481732845306396
Epoch [31/100], Loss: 0.2788243889808655
Epoch [41/100], Loss: 0.2682393491268158
Epoch [51/100], Loss: 0.2612352967262268
Epoch [61/100], Loss: 0.25660133361816406
Epoch [71/100], Loss: 0.25353553891181946
Epoch [81/100], Loss: 0.25150686502456665
Epoch [91/100], Loss: 0.2501640319824219
